In [1]:
import Data
import regression
import dclassifier
import numpy as np
import pandas as pd
import backtest
import Genetic_Algorithm_2

In [2]:
training_months = [pd.Period('2019-04', 'M'), pd.Period('2019-05', 'M'), pd.Period('2019-06', 'M'), pd.Period('2019-07', 'M')]
testing_month = [pd.Period('2019-04', 'M'), pd.Period('2019-08', 'M'), pd.Period('2019-09', 'M'), pd.Period('2019-10', 'M')]
data = Data.Data()

prices = data.prices
prices = prices[prices.index.to_period('M').isin(training_months)]

In [3]:
prices

DATE_TIME,AUDUSD,EURCHF,EURJPY,EURUSD,USDCAD,USDCHF,USDJPY
2019-04-01 00:00:00,0.71161,1.11729,124.660,1.12258,1.33485,0.99543,111.042
2019-04-01 00:15:00,0.71122,1.11737,124.656,1.12281,1.33475,0.99509,111.044
2019-04-01 00:30:00,0.71126,1.11734,124.622,1.12299,1.33478,0.99483,110.969
2019-04-01 00:45:00,0.71107,1.11726,124.574,1.12287,1.33500,0.99485,110.934
2019-04-01 01:00:00,0.71090,1.11699,124.504,1.12258,1.33457,0.99501,110.913
...,...,...,...,...,...,...,...
2019-07-31 22:45:00,0.68377,1.10083,120.416,1.10693,1.31983,0.99425,108.784
2019-07-31 23:00:00,0.68356,1.10062,120.363,1.10629,1.31998,0.99459,108.797
2019-07-31 23:15:00,0.68398,1.10040,120.422,1.10677,1.31976,0.99400,108.803
2019-07-31 23:30:00,0.68458,1.10057,120.441,1.10748,1.31878,0.99345,108.752


In [10]:
thresholds = np.arange(0.0001, 0.0010, 0.0001)
currencies = ['EURCHF']

# remove AUDUSD, because it frequently creates a regression expression that doesn't have DCl
#currencies = [curr for curr in currencies if curr != 'AUDUSD']

combinations = [(threshold, currency) for threshold in thresholds for currency in currencies]

losses = {}
best_individual = {}
for combination in combinations:
    threshold, currency = combination

    events, dcclass = dclassifier.classify_split_timeseries(prices[currency], threshold)
    DCs_with_OSs = [(a['end'] - a['start'], b['start'] - a['end']) for (_, a), (_, b) in zip(events.iloc[:-1].iterrows(), events.iloc[1:].iterrows()) if b['prev_os']]
    
    loss, individual = regression.SRGP(DCs_with_OSs)

    best_individual[combination] = individual
    losses[combination] = loss

In [11]:
losses

{(0.0001, 'EURCHF'): 2.6450487528833713,
 (0.0002, 'EURCHF'): 4.130745173062754,
 (0.00030000000000000003, 'EURCHF'): 6.03347393707353,
 (0.0004, 'EURCHF'): 7.94897830275916,
 (0.0005, 'EURCHF'): 10.564910749922259,
 (0.0006000000000000001, 'EURCHF'): 11.876451300409029,
 (0.0007000000000000001, 'EURCHF'): 13.857956940027131,
 (0.0008, 'EURCHF'): 16.968015624933567,
 (0.0009000000000000001, 'EURCHF'): 20.858188601209754}

In [13]:
def print_node(node):
    string = ''
    if node.val:
        string += (node.val if isinstance(node.val, str) else str(round(node.val, 3)))
    if len(node.children):
        string += f" ({' , '.join([print_node(child) for child in node.children])})"
    return string

In [14]:
print_node(best_individual[(0.0001, 'EURCHF')])

'- (log (- (DCl , -9.177)) , sin (cos (9.983)))'

In [4]:
class e_handler:
    def __init__(self, classified_data, theta) -> None:
        #, regressor
        self.classified_data = classified_data
        self.dc_count = 0
        self.detector = None
        self.theta = theta
        #self.regressor = regressor

    def reset(self):
        self.dc_count = 0
        self.detector = None
        
    
    def step(self, price, time):
        if self.detector == None:
            self.detector = dclassifier.DC_EVENT_DETECTOR(self.theta, time, price)
            return [-1, 0]
        
        self.detector.step(price, time)

        #print(len(self.classified_data) , self.detector.get_dc_count())
        
        # if no new dc_event happend => return 0
        if self.dc_count == self.detector.get_dc_count():
            return [-1, 0]

        # if new dc_event happend
        # Will be replaced by the classification algortihm
        if self.detector.get_dc_count() == len(self.classified_data) or (self.detector.get_dc_count() - 1) == len(self.classified_data):
            return [-1, 0]
        
        if not self.classified_data.iloc[self.detector.get_dc_count()]["prev_os"]:
            self.dc_count += 1
            return [-1, 0]
        #
        
        #dc_data = self.detector.get_dc_data
        os_end = self.classified_data.iloc[self.detector.get_dc_count()]["start"]
        os_start = self.classified_data.iloc[self.detector.get_dc_count() - 1]["end"]
        #os_length = regression.eval_tree(self.regressor, dc_data['event_time'])

        self.dc_count += 1
        
        
        return [os_end-os_start, 'Up' if self.classified_data.iloc[self.detector.get_dc_count() - 1]['event_price'] > 0 else 'Down']

        #return [os_length, 'Up' if dc_data['event_price'] >= 0 else 'Down']

In [7]:
thresholds = [0.0001, 0.0002, 0.0003, 0.0004, 0.00005]
handlers = []
currency = "EURCHF"
for el in thresholds:
    events, dcclass = dclassifier.classify_split_timeseries(prices[currency], el)
    handlers.append(e_handler(events, el))

    #print(events)


backtesting_env = backtest.Optimize_eps(prices[currency], thresholds, handlers)

In [8]:
handlers

In [7]:
""" 
time = 0
for el in prices[currency]:
    for i in range(len(handlers)):
        print(handlers[i].step(el, time))
    time += 1 
"""

' \ntime = 0\nfor el in prices[currency]:\n    for i in range(len(handlers)):\n        print(handlers[i].step(el, time))\n    time += 1 \n'

In [15]:
l = backtesting_env.evaluate([0.1, 0.4, 0.2, 0.1, 0.2])

vol 0.000253204827932316 sharpe -7.2689365479126975 ann_ret: 0.0007824947692336437 return 0.026565581346474354 capital 100000


In [16]:
backtesting_env.hist

0       100000.000000
1        99992.840330
2        99995.525086
3       100002.685140
4       100026.857895
            ...      
8443    102656.558135
8444    102656.558135
8445    102656.558135
8446    102656.558135
8447    102656.558135
Length: 8448, dtype: float64

In [9]:
best = Genetic_Algorithm_2.genetic_algorithm(prices[currency], thresholds, backtesting_env, population_size=50, generations=5)

vol 0.00024493203801719004 sharpe -6.6474571203303 ann_ret: 0.004153519660436977 return 0.14905168484950088 capital 100000
vol 0.0002428150576738134 sharpe -6.5992133659372145 ann_ret: 0.004562873151106039 return 0.16485766454205805 capital 100000
vol 0.00023912473785051884 sharpe -6.641050040377612 ann_ret: 0.004790656846967112 return 0.1737439937715385 capital 100000
vol 0.0002256724489016728 sharpe -6.832716766158934 ann_ret: 0.005522208534668893 return 0.20273043807742827 capital 100000
vol 0.0002488057650381334 sharpe -6.730017424808951 ann_ret: 0.003418658307729139 return 0.12119765876641643 capital 100000
vol 0.00025544362807282715 sharpe -7.208372245544382 ann_ret: 0.0007697486341837756 return 0.0261274164985698 capital 100000
vol 0.0002605841903489442 sharpe -7.1402392973999085 ann_ret: 0.0004633592454297286 return 0.015649318972674228 capital 100000
vol 0.0002624655828026807 sharpe -7.073621077050054 ann_ret: 0.0005276731900469844 return 0.017840123917882155 capital 100000
vo

/home/linus/ghq/github.com/linux-03/MTDC/backtest.py:29: RuntimeWarning: invalid value encountered in scalar divide
  W = np.dot(weights, rev_points)/np.sum(weights)
/home/linus/ghq/github.com/linux-03/MTDC/backtest.py:163: RuntimeWarning: divide by zero encountered in scalar divide
  return (ret - self.rf_rate)/vol


vol 0.0 sharpe -inf ann_ret: 0.0 return 0.0 capital 100000
vol 0.0002597466526126156 sharpe -7.2206380016022305 ann_ret: 0.00022678027311040516 return 0.007629736166637304 capital 100000
vol 0.00023984947583550542 sharpe -7.675623896899966 ann_ret: 0.0007751204171457804 return 0.026312056348240003 capital 100000
vol 0.0002641048096614941 sharpe -7.022533779481292 ann_ret: 0.0005577892164583265 return 0.018867578551010084 capital 100000
vol 0.00025558482190859847 sharpe -6.988110656735572 ann_ret: 0.0016472555720483673 return 0.056720577846897505 capital 100000
vol 0.0002488057650381334 sharpe -6.730017424808951 ann_ret: 0.003418658307729139 return 0.12119765876641643 capital 100000
vol 0.0002567612631839749 sharpe -7.167099851264635 ann_ret: 0.0007871970633896996 return 0.026727274609633315 capital 100000
vol 0.00027035719857273284 sharpe -6.874735607258378 ann_ret: 0.00049509582355034 return 0.01672982867253988 capital 100000
vol 0.0002654622780596553 sharpe -7.001441724135622 ann_ret